In [ ]:
import pandas as pd
from utils import (
    df_to_excel_col_dict,
    getRowSharesIdx,
    load_hh_raw,
    load_person_assign_day,
    out_dir,
    prep_data_2d,
    tour_extract_wkday_dir,
    write_to_excel,
)
from xlsxwriter.utility import xl_rowcol_to_cell

In [ ]:
out_filepath = out_dir / "11_Person_Attr.xlsx"

In [ ]:
hh = load_hh_raw(income=True, home_county=True)
person = (
    load_person_assign_day(
        tour_extract_wkday_dir / "person-assign_day.csv", demographics=True
    )
    .join(hh, on="hhno", how="left")
    .to_pandas()
)

In [ ]:
writer = pd.ExcelWriter(out_filepath, engine="xlsxwriter")
workbook = writer.book
format1 = workbook.add_format({"num_format": "#,##0.0"})

In [ ]:
wt_cols = ["count", "psexpfac"]
wt_desc = ["(Unweighted)", "(Weighted)"]

d1_dict = df_to_excel_col_dict["home_county"]
for key in ["raceeth", "hh_income_broad_cat", "pagey", "pgend"]:
    d2_dict = df_to_excel_col_dict[key]
    row = 0
    sname = d2_dict["desc"]

    for wc, wd in zip(wt_cols, wt_desc):
        title = "Persons by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        tab = prep_data_2d(
            person,
            d1_dict["col"],
            d1_dict["vals"],
            d1_dict["labels"],
            d2_dict["col"],
            d2_dict["vals"],
            d2_dict["labels"],
            wc,
        )
        row = write_to_excel(writer, tab.astype("float64"), sname, title, row)

        title = (
            "Row Shares by " + d2_dict["desc"] + " and " + d1_dict["desc"] + " " + wd
        )
        row = write_to_excel(writer, getRowSharesIdx(tab.copy()), sname, title, row)

    tab_range = xl_rowcol_to_cell(row, 1) + ":" + xl_rowcol_to_cell(row, tab.shape[1])
    _ = writer.sheets[sname].set_column(tab_range, 11, format1)

In [ ]:
writer.close()